---
title: SELECT statement
tasl:
    source: fa2023\lectures\lect13-sql-selects.qmd
    tags:
    - lecture
    - lecture-13
    topic: SELECT statement
---


# Getting to SELECT

## Getting to Select

:::: {.columns}
::: {.column}

### Database access workflow

1. Retrieving credentials into your program.  *No hardcoding!*

1. Connecting to the database using a *local database engine.*

1. Through the life of the program:

    a. Sending the *query* to the DBMS through the connection.
    a. Retriving the *result set*

1. Finally, closing the connection.
:::
::: {.column}
### Key terms
<p>&nbsp;</p>
credentials
: host, username, password, database

local database engine
: software on your machine that manages the connection through the internet
to your remote server.

query
: a command sent to the remove database server

result set
: the package returned from the remote server.  It may contain
data or error information.

:::
::::

## Database connection

:::: {.columns}
::: {.column width=60%}

In [ ]:
#| eval: true
#| echo: fenced
import os
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine,text
# load credentials from .env file
load_dotenv()
# store these credentials a dictionary for later reference
config = {
  'user': os.getenv("HW5_USER"),
  'password': os.getenv("HW5_PASSWORD"),
  'host': os.getenv("HW5_HOST"),
  'database': os.getenv("HW5_DB_NAME")
}
engine_uri = f"mysql+pymysql://{config['user']}:{config['password']}@{config['host']}/{config['database']}"

# create a database connection.  THIS IS THE ACTUAL CONNECTION!
cnx = create_engine(engine_uri)

:::
::: {.column width=40%}
### Connection block

This single block of code general sits at the top of all your QMD files.

This block of code creates the `cnx` variable - the actually connection
to the database.

We'll use `cnx` later in the code whenever we want to talk with
the DBMS.
:::
::::

## Database connection - Imports

:::: {.columns}
::: {.column width=60%}

In [ ]:
#| eval: true
#| echo: fenced
#| code-line-numbers: 4-7
import os
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine,text
# load credentials from .env file
load_dotenv()
# store these credentials a dictionary for later reference
config = {
  'user': os.getenv("HW5_USER"),
  'password': os.getenv("HW5_PASSWORD"),
  'host': os.getenv("HW5_HOST"),
  'database': os.getenv("HW5_DB_NAME")
}
engine_uri = f"mysql+pymysql://{config['user']}:{config['password']}@{config['host']}/{config['database']}"

# create a database connection.  THIS IS THE ACTUAL CONNECTION!
cnx = create_engine(engine_uri)

:::
::: {.column width=40%}
### Imports

The `import` commands load modules into your *python* program. 

These modules must first be installed in your virtual environment using 
`poetry` or `pip` to install the necessary modules.  I prefer `poetry`!

`dotenv` module is used to load environment variable from disk into memory.

`sqlalchemy` module provides many different tools for connecting to, managing, and querying databases.
`sqlalchemy` is known as an *Object Relational Mapper* or *ORM*.  When used to it's full extent
it allows to you seamlessly load and store objects without having to think about the SQL.

But that kind of defeats the purpose for this class!  So we'll only use a few of the tools provided.

:::
::::

## Database connection - Load .env into environment

:::: {.columns}
::: {.column width=60%}

In [ ]:
#| eval: true
#| echo: fenced
#| code-line-numbers: 8-16
import os
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine,text
# load credentials from .env file
load_dotenv()
# store these credentials a dictionary for later reference
config = {
  'user': os.getenv("HW5_USER"),
  'password': os.getenv("HW5_PASSWORD"),
  'host': os.getenv("HW5_HOST"),
  'database': os.getenv("HW5_DB_NAME")
}
engine_uri = f"mysql+pymysql://{config['user']}:{config['password']}@{config['host']}/{config['database']}"

# create a database connection.  THIS IS THE ACTUAL CONNECTION!
cnx = create_engine(engine_uri)

:::
::: {.column width=40%}
### Getting your connection variables.

Rather than hard-coding your database connection into your code, we'll
store the connection information on your disk and load it when your program starts.

`load_dotenv()` searches for a `.env` file and if found, loads the contents into your environment.

`os.getenv()` searches for a *key* in the environnment and if found returns it's *value*.

A database connection is defined by four attributes:
  * Server name
  * Database name
  * Username
  * Password

To minimize pollution in the namespace we'll store these four separate variables in a single dictionary for later reference.

:::
::::

## Database connection - Create connection

:::: {.columns}
::: {.column width=60%}

In [ ]:
#| eval: true
#| echo: fenced
#| code-line-numbers: 17-20
import os
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine,text
# load credentials from .env file
load_dotenv()
# store these credentials a dictionary for later reference
config = {
  'user': os.getenv("HW5_USER"),
  'password': os.getenv("HW5_PASSWORD"),
  'host': os.getenv("HW5_HOST"),
  'database': os.getenv("HW5_DB_NAME")
}
engine_uri = f"mysql+pymysql://{config['user']}:{config['password']}@{config['host']}/{config['database']}"

# create a database connection.  THIS IS THE ACTUAL CONNECTION!
cnx = create_engine(engine_uri)

:::
::: {.column width=40%}
### Creating the connection

Armed with our connection varaibles we create a connection string and open a connection to the database using a *local database engine.*

A connection to ANY database and server starts with a [*connection string*](https://www.connectionstrings.com/){target="_blank"}

Through the connection string you can customize the connection and how information passes through it.

Connection strings are used by your *local database engine* to *locate the database host*, *open the appropriate port*, 
*pass credentials like username and password*, and finally *select the database to use*.

Connection strings also manage which languages and character sets are used, sizes of transmission buffers, and more.

*RTFM* for your [*local database engine*!](https://docs.sqlalchemy.org/en/20/core/engines.html){target="_blank"}

:::
::::

## Another helper block

In my code I created another block with a function.

Normally, you'd set `echo: false` to not display the code.


In [ ]:
#| echo: fenced
from tabulate import tabulate
from IPython.display import display, Markdown

def display_dataframe_as_table( df, width="100%" ):
    """ Display dataframe in pretty format for Quarto """
    markdown_table = tabulate(df, headers='keys',showindex=False)
    html_table = markdown_table
    display(Markdown(html_table))

## Select statement

:::: {.columns}
::: {.column width=50%}
A *SELECT* query is the one of the most fundamental
statements in SQL.

A *SELECT* statement describe a request to perform operations within the database and return a *result set*.  

This *result set* is a relation.

This *result* can be used in subsequent operations or stored back into the DB as a new table.

A SQL query is relational algebra:

$\Pi_{A_1, A_2, ... A_n}(\sigma_{P}(R_1 \times R_2 \times ... \times R_n) )$

:::
::: {.column width=50%}
**SELECT** A1, A2, A3

**FROM** R1,R2

**WHERE** P

**ORDER BY** A2,A3,A1;

* A* represents an attribute (column), a literal, a function, or an operation
* R* represents a relation (table), cartesian project, or join
* P is a predicate, conditions or filters

:::
::::

## SELECT clause and examples

:::: {.columns}
::: {.column width=50%}

* The select clause lists the attributes desired in the result of a query, corresponds to the projection operation of the relational algebra

* SQL names are case insensitive

* SQL allows duplicates in relations as well as in query results

* To force the elimination of duplicates, use the keyword distinct

* An asterisk in the select clause denotes “all attributes”

* May rename columns using alias

:::
::: {.column width=50%}

```{sql}
-- this is syntax
SELECT [distinct] {*, column [[as] alias], ...}
FROM table {or tables}

-- this is a real statement (and comment!)
SELECT
  *
FROM
  REGIONS

-- indents are not managed
-- comments stop processing of line

```

:::
::::

## *Regions* table

::: {style="font-size: 50%;"}

In [ ]:
#| echo: false
result = pd.read_sql("""
select * from regions
""",cnx)
result.head(20)

:::

## SELECT clause and examples

:::: {.columns}
::: {.column width=50%}
### Using * to select all columns

In [ ]:
#| echo: fenced
#| code-line-numbers: 4
result = pd.read_sql("""
select * from regions
""",cnx)
result

:::
::: {.column width=50%}
### Selecting only one attribute

In [ ]:
#| echo: fenced
#| code-line-numbers: 4-7
result = pd.read_sql("""
select
  region_name
from
  regions
""",cnx)
result

:::
::::


## *Employees* table - first 20 rows

::: {style="font-size: 50%;"}

In [ ]:
#| echo: false
result = pd.read_sql("""
select * from employees
""",cnx)
result.head(20)

:::

## SELECT clause and examples

:::: {.columns}
::: {.column width=50%}
### Gathering all last names

In [ ]:
#| echo: fenced
#| code-line-numbers: 4
result = pd.read_sql("""
select last_name from employees
""",cnx)
result

:::
::: {.column width=50%}
### Eliminating duplicates with DISTINCT

In [ ]:
#| echo: fenced
#| code-line-numbers: 4
result = pd.read_sql("""
select distinct last_name from employees
""",cnx)
result

:::
::::

## SELECT clause and examples

:::: {.columns}
::: {.column width=50%}
### Combining last and first names

In [ ]:
#| echo: fenced
#| code-line-numbers: 4-5
result = pd.read_sql("""
select concat(last_name,',',first_name)
from employees
""",cnx)
result.head(7)

:::
::: {.column width=50%}
### Then renaming the column

In [ ]:
#| echo: fenced
#| code-line-numbers: 4-5
result = pd.read_sql("""
select concat(last_name,',',first_name) as full_name
from employees
""",cnx)
result.head(7)

:::
::::

## SELECT clause and examples

:::: {.columns}
::: {.column width=50%}
### attribute can be literal or function

In [ ]:
#| echo: fenced
#| code-line-numbers: 4-8
result = pd.read_sql("""
select
  27 as "Literal",
  now() as "Current Time" 
from
  dual
""",cnx)
result.head(7)

<p>&nbsp;</p>
Note that we're using a built-in table named *dual* that 
returns what you put in as a table that can be used
in subsequent operations.

:::
::: {.column width=50%}
### Doing computations

The attribute can be a function or arithmetic operation

In [ ]:
#| echo: fenced
#| code-line-numbers: 4-6
result = pd.read_sql("""
select concat(last_name,',',first_name) as full_name,
salary*12 as "Annual Salary"
from employees
""",cnx)
result.head(5)

:::
::::

## WHERE clause and examples

:::: {.columns}
::: {.column width=50%}

* The WHERE clause specifies conditions that the results must satisfy, corresponding to the $\sigma$ predicate in relational algebra.

* An individual WHERE element returns TRUE or FALSE for every row.  This is applied during the query and will return only rows that evaluate TRUE in the where clause.

* Attributes in where clause can be compared using relational operators: <  <=  =  >=  >
* WHERE elements can be combined using *AND*, *OR* and *NOT* logical operators.

* WHERE also uses special operators: *BETWEEN*, *IN* and *IS NULL*

:::
::: {.column width=50%}

In [ ]:
#| echo: fenced
#| code-line-numbers: 5-10
result = pd.read_sql("""
-- note that this is a monthly salary!
SELECT
  last_name, first_name, salary
FROM
  employees
WHERE
  salary > 10000
""",cnx)
result.head(5)

:::
::::




## WHERE clause and examples

:::: {.columns}
::: {.column width=50%}
### Selecting a department

In [ ]:
#| echo: fenced
#| code-line-numbers: 4-6
result = pd.read_sql("""
select last_name,department_id
from employees
where department_id=110;
""",cnx)
result.head(7)

<p>&nbsp;</p>
Data types matter.  You'll get an error
if you set *department_id='110'*

:::
::: {.column width=50%}
### Using BETWEEN
The attribute can be a function or arithmetic operation

In [ ]:
#| echo: fenced
#| code-line-numbers: 4-6
result = pd.read_sql("""
select concat(last_name,',',first_name) as full_name, salary
from employees
where salary BETWEEN 10000 and 12000;
""",cnx)
result.head(5)

:::
::::

## WHERE clause and examples

:::: {.columns}
::: {.column width=50%}
### Using IN

In [ ]:
#| echo: fenced
#| code-line-numbers: 4-6
result = pd.read_sql("""
select last_name,department_id
from employees
where department_id in (100,145,146)
""",cnx)
result.head(7)

<p>&nbsp;</p>

:::
::: {.column width=50%}
### The long way

In [ ]:
#| echo: fenced
#| code-line-numbers: 4-6
result = pd.read_sql("""
select last_name,department_id
from employees
where department_id=30 or department_id=70 or department_id=120
""",cnx)
result

<p>&nbsp;</p>

:::
::::

## WHERE clause and examples

:::: {.columns}
::: {.column width=50%}
### What are all the department IDs?

In [ ]:
#| echo: fenced
#| code-line-numbers: 4-6
result = pd.read_sql("""
select distinct department_id
from employees
order by department_id
""",cnx)
result

<p>&nbsp;</p>

:::
::: {.column width=50%}
### Who is missing a manager?

In [ ]:
#| echo: fenced
#| code-line-numbers: 4-6
result = pd.read_sql("""
select last_name,job_id,manager_id
from employees
where manager_id is NULL
""",cnx)
result.head(7)

<p>&nbsp;</p>

NULL must be all caps!!

:::
::::

## Computations

:::: {.columns}
::: {.column width=50%}
### Calculating commissions

In [ ]:
#| eval: false
#| echo: fenced
#| code-line-numbers: 5-13
result = pd.read_sql("""
select 
  concat(last_name,',',first_name) as "Name",
  salary as "Monthly",
  12.0*salary+commission_pct*salary as "With/Commission"
from
  employees
order by
  last_name,
  first_name
""",cnx)
result

<p>&nbsp;</p>
What are the NaN "not a number" entries?

How do we remove the NaN (not a number) from
list? 

:::
::: {.column width=50%}
### Calculating commissions

In [ ]:
#| echo: false
#| eval: true
result = pd.read_sql("""
select 
  concat(last_name,',',first_name) as "Name",
  salary as "Monthly",
  12.0*salary+commission_pct*salary as "With/Commission"
from employees
order by last_name, first_name
""",cnx)
result

:::
::::

## IFNULL operator

:::: {.columns}
::: {.column width=50%}
### The IFNULL operator

IFNULL( expr1, expr2 ) replaces NULL with a value.

* IF expr1 is NOT NULL then returns expr1

* IF expr1 IS NULL, then returns expr2.

:::
::: {.column width=50%}
### Calculating commissions

In [ ]:
#| eval: false
#| echo: fenced
#| code-line-numbers: 5-10
result = pd.read_sql("""
select 
  concat(last_name,',',first_name) as "Name",
  salary as "Monthly",
  IFNULL(12.0*salary+commission_pct*salary,0.0) as "With/Commission"
from employees
order by last_name, first_name
""",cnx)
result

:::
::::

## IFNULL operator

:::: {.columns}
::: {.column width=50%}
### IFNULL( expr1, 0.0)

In [ ]:
#| echo: false
#| eval: true
result = pd.read_sql("""

select 
  concat(last_name,',',first_name) as "Name",
  salary as "Monthly",
  IFNULL(12.0*salary+commission_pct*salary,0.0) as "With/Commission"
from
  employees
order by
  last_name,
  first_name

""",cnx)
result

:::
::: {.column width=50%}
### IFNULL( expr1, 'missing')

In [ ]:
#| echo: false
#| eval: true
result = pd.read_sql("""
select 
  concat(last_name,',',first_name) as "Name",
  salary as "Monthly",
  IFNULL(12.0*salary+commission_pct*salary,'missing') as "With/Commission"
from employees
order by last_name, first_name
""",cnx)
result

:::
::::

## String operations

The operator *LIKE* uses patterns (case insensitive) (use *LIKE BINARY* for case sensitive) for string-matching operations using two special characters:

    * percentage ( % ) matches any substring (none or many characters)
    * underscore ( _ ) matches any single character

Examples:

'Intro%' 		matches any string beginning with “Intro”

'%Comp%' 	matches any string containing “Comp” as a substring

'_ _ _' 		matches any string of exactly three characters

'_ _ _ %' 		matches any string of at least three characters

'%_ a _ ' 		same as before but the second to the last letter is 'a'

## String operations

:::: {.columns}
::: {.column width=50%}
### DOUBLE %% when using Python

In [ ]:
#| eval: true
#| echo: fenced
#| code-line-numbers: 5-6
result = pd.read_sql("""
select last_name from employees
where last_name like 'MC%%'
""",cnx)
result

:::
::: {.column width=50%}
### DOUBLE %% when using Python

In [ ]:
#| eval: true
#| echo: fenced
#| code-line-numbers: 5-6
result = pd.read_sql("""
SELECT phone_number FROM employees
WHERE phone_number LIKE '%%123%%';
""",cnx)
result

:::
::::

## ORDER BY clause

:::: {.columns}
::: {.column width=50%}
*ORDER BY* is used to specify
the sort order in the result set.

The result set is sorted by the first attribute listed in the *ORDER BY*.

If there is a tie, *ORDER BY* moves to the second attribute, and so on.

Attribute expression may be modified with ASC (the default) or DESC (for descending)

Attributes in *ORDER BY* can be computed!

:::
::: {.column width=50%}
### Example

In [ ]:
#| eval: true
#| echo: fenced
#| code-line-numbers: 5-10
result = pd.read_sql("""
select 
  concat(last_name,',',first_name) as "Full Name", salary as "Monthly"
from employees
order by
  salary desc,
  last_name, first_name
""",cnx)
result

:::
::::

## SELECT summary

:::: {.columns}
::: {.column}
* `SELECT` - variable to project
* `FROM` - tables to combine
* `WHERE` - filters to apply
* `ORDER BY` - order of results

* `CONCAT` - combining columns
* `AS` - Renaming columns
* Literals
* Computations in SELECT

<p>&nbsp;</p>

See the [MySQL *SELECT* documentation](https://dev.mysql.com/doc/refman/8.0/en/select.html).

:::
::: {.column}
* `WHERE`
    * AND, OR, NOT
    * `<` `<=` `=` `=>` `>`

* `BETWEEN` - for *where*
* `IN` - checking set membership
* `NULL` values
* `IFNULL` operator
* String matches `%%`

<p>&nbsp;</p>

See the [MySQL *Functions* documentation](https://dev.mysql.com/doc/refman/8.0/en/functions.html)

:::
::::
